
1. load required packages 

2. load calibration files; .poni defining geometry and mask

3. define integrator function



In [ ]:
import pyFAI
import baseline
import fabio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pygix as pg
import mplcursors
from scipy.interpolate import interp1d
from scipy import sparse
from scipy.signal import find_peaks
from scipy.sparse.linalg import spsolve
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
import csv
import os

In [ ]:
# load the calibration files and visualise mask

poni_file = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/calib/MZ_AgBh_calib.poni"
calibration = pyFAI.load(poni_file) 

maskfile = "/home/goughl/Documents/beamtime_data/2409_DLS/ming/calib/MZ_mask.edf"
mask = fabio.open(maskfile).data

# uncomment to plot mask

mask = np.logical_not(mask).astype(int)
mask = mask
# plt.imshow(mask, cmap='gray')
# plt.colorbar(label="Mask (1 = unmasked, 0 = masked)")
# plt.show()

In [ ]:
# # defining all functions to be used
# # shouldn't need to change these

# integrator function with arguments defined in pyFAI documentation
integrator = AzimuthalIntegrator(
    detector=calibration.detector,
    wavelength=calibration.wavelength,
    dist=calibration.dist,  
    poni1=calibration.poni1,
    poni2=calibration.poni2,
    rot1=calibration.rot1,
    rot2=calibration.rot2,
    rot3=calibration.rot3,
)

In [ ]:
# #plot the raw tif image

# image = fabio.open("/home/goughl/Documents/beamtime_data/2304_DLS/raw_data/230429-3-MINERVA_DCV5T_HT/long_exposure/p2mImage1019682.tif")  # Replace with the actual path to your image file
# data = image.data  

# vmin = 100  # min intensity value
# vmax = 15000  # max intensity value

# # for long exposure images (t ~ 30 s) vmax ~ 15000
# # for short exposure images (t ~ 3 s) vmax ~ 1000

# plt.imshow(data, cmap='gray', vmin=vmin, vmax=vmax)
# plt.colorbar()
# plt.xlabel('Pixel X')
# plt.ylabel('Pixel Y')
# plt.title('Raw 2D Detector Image')
# plt.show()

In [ ]:
# # check you're in the right folder 
# # lists all the files in a directory and puts them in numerical roder

# folder_path = r"/home/goughl/Documents/beamtime_data/2112_ESRF/MAR/ANDREAS/paper"
# files = [filename for filename in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, filename)) and filename.endswith('.edf')]
# #sorted_files = sorted(files, key=lambda x: int(x.split("p2mImage")[1].split(".")[0]))

# # for filename in sorted_files:
# #     print(filename)

# print(files)

In [ ]:
## plot the detector image with mask applied

# detector_image_file = "/home/goughl/Documents/beamtime_data/2304_DLS/raw_data/230429-3-MINERVA_DCV5T_HT/long_exposure/p2mImage1019682.tif"  # Replace with the path to your detector image file
# detector_image = fabio.open(detector_image_file).data

data_file = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/MZ_1p2mImage1298544.tif"
# mask_file = "/home/goughl/Documents/beamtime_data/2304_DLS/calib/mask_AgBh.edf" # Replace with the path to your mask file
# mask_data = fabio.open(mask_file).data

# inverted_mask = ~mask_data
data = fabio.open(data_file).data
masked_image = np.multiply(mask,data)

# set the intensity range (vmin and vmax)
vmin = 0  # Minimum intensity value
vmax = 50  # Maximum intensity value

# Plot the masked detector image with specified intensity range
# image = plt.imread(//image_path)

plt.imshow(masked_image, cmap='gray', vmin=vmin, vmax=vmax)
plt.colorbar()
plt.title("Masked Detector Image")
plt.show()

In [ ]:
data_masked

In [ ]:
# plot the integrated data (individual)

data_file = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/MZ_1p2mImage1298544.tif"
data = fabio.open(data_file).data
data_masked = np.multiply(mask,data)
# uncomment the line above to mask



q, I = integrator.integrate1d(data_masked,
                              1000, 
                              unit="q_A^-1", 
                              azimuth_range=(160,180), 
                              radial_range=(0.25, 2.25), 
                              polarization_factor=0.99, 
                              )

# 235, 255 is out of plane i think??
# 180, 200 in plane? not 100% sure

plt.plot(q, I)
plt.xlabel("q (1/Å)")
plt.ylabel("Intensity (a.u.)")
plt.title("1D integration in-plane")
plt.grid(True)
plt.show()


In [ ]:
# 1D integrations of all files in a directory
# not normalised

folder_path = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/"
output_folder = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/out"


qs = []
Is = []
labels = []

for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):
        data = np.multiply(mask,data)
        data = fabio.open(os.path.join(folder_path, filename)).data
        q, I = integrator.integrate1d(data,
                                      1000,
                                      correctSolidAngle=True,
                                      unit="q_A^-1",
                                      azimuth_range=(102, 117),
                                      radial_range=(0.2, 2),
                                      polarization_factor=0.99)
        label = filename 
        qs.append(q)
        Is.append(I)
        labels.append(label)
        
        # for Ming samples ; 163,178 for IP,  for OoP
    

# save = False
# # change to true when you want to save the files (unnormalised!!)

# if save == True:
#     for filename, q, I, label in zip(os.listdir(folder_path), qs, Is, labels):
#         data = {'q': q, 'Intensity': I}
#         df = pd.DataFrame(data)
#         base_filename = os.path.splitext(filename)[0]  
#         output_filename = os.path.join(output_folder, f"1D_int_RT_45_80_{label}.csv") 
#         df.to_csv(output_filename, index=False)

#         plt.plot(q, I, label=f"{label}")        
# else:

####  if you want to save the unnormalised integrations, tab the for loop to the same level if if statement
for q, I, label in zip(qs, Is, labels):  
        plt.plot(q, I, label=f"{label}")

plt.xlabel("q (1/Å)")
plt.ylabel("Intensity (a.u.)")
#plt.title("")

plt.legend()
plt.show()

In [ ]:
# normalising to a chosen point in q space, where you expect to see nothing

background_q = 1.1# choose a q value where there is only background
bg_range = 0.1# choose a small range of which background_q is the centre to average


normalized_intensities = []
peak_coordinates = []


for i, (q, I) in enumerate(zip(qs, Is)):
    max_peak = min(I[(q > background_q - bg_range) & (q < background_q + bg_range)])
    max_index = np.argmin(I[(q > background_q - bg_range) & (q < background_q + bg_range)])
    q_max_peak = q[(q > background_q - bg_range) & (q < background_q + bg_range)][max_index]

    normalized_intensity = I / max_peak
    normalized_intensities.append(normalized_intensity)
    peak_coordinates.append((q_max_peak, max_peak))

    plt.plot(q, normalized_intensity, label=f"{labels[i]} nm")

plt.xlabel("q (1/Å)")
plt.ylabel("Normalized intensity")
#plt.title("Integration in plane \n  DCV5T:C60")
plt.legend()
plt.show()

# # Print the coordinates of the normalizing peaks
# for i, (q_max_peak, max_peak) in enumerate(peak_coordinates):
#     print(f"Peak {i + 1} coordinates: q = {q_max_peak:.4f}, I = {max_peak:.4f}")


In [ ]:
### save normalised data in csv

# specify the directory where you want to save the csv files
save_dir = r"/home/goughl/Documents/beamtime_data/2409_DLS/ming/out"
os.makedirs(save_dir, exist_ok=True)

# normalising to a chosen point in q space, where you expect to see nothing
background_q = 1.1 # choose a q value where there is only background
bg_range = 0.1 # choose a small range of which background_q is the centre to average

normalized_intensities = []
peak_coordinates = []

for i, (q, I) in enumerate(zip(qs, Is)):
    max_peak = min(I[(q > background_q - bg_range) & (q < background_q + bg_range)])
    max_index = np.argmin(I[(q > background_q - bg_range) & (q < background_q + bg_range)])
    q_max_peak = q[(q > background_q - bg_range) & (q < background_q + bg_range)][max_index]

    normalized_intensity = I / max_peak
    normalized_intensities.append(normalized_intensity)
    peak_coordinates.append((q_max_peak, max_peak))

    original_filename = os.path.splitext(os.path.basename(labels[i]))[0]

    filename = os.path.join(save_dir, f"1D_OoP_102_117_{original_filename}.csv")
    
    # naming convention: {YYMMDD}_{project}_{sample}_{IP or OoP}_{azimuth_range1}_{azimuth_range2}_{normalised or not}
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['q', 'Intensity'])
        for q_value, intensity in zip(q, normalized_intensity):
            writer.writerow([q_value, intensity])

    plt.plot(q, normalized_intensity, label=f"{labels[i]}")  # Label each dataset with its filename

plt.xlabel("q (1/Å)")
plt.ylabel("Normalized intensity")
#plt.title("")
plt.legend()
plt.show()

# for i, (q_max_peak, max_peak) in enumerate(peak_coordinates):
#     print(f"Peak {i + 1} coordinates: q = {q_max_peak:.4f}, I = {max_peak:.4f}")
